In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np


In [78]:
#importing dataset
powell_data = pd.read_csv('./data/powell_weather.csv')
powell_data.head()

,STATION,NAME,DATE,AWND,PRCP,SNWD,TMAX,TMIN,FOG,THUNDER,HAZE
0,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/1/97,NaN,NaN,NaN,95,66,NaN,NaN,NaN
1,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/2/97,NaN,NaN,NaN,93,67,NaN,NaN,NaN
2,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/3/97,NaN,NaN,NaN,90,65,NaN,NaN,NaN
3,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/4/97,NaN,NaN,NaN,94,62,NaN,NaN,NaN
4,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/5/97,NaN,NaN,NaN,89,63,NaN,NaN,NaN


In [79]:
#Data preprocessing and filling null values with 0 to indicate not recorded or did not happen depending on the column
columns_to_update = ['FOG', 'THUNDER', 'HAZE', 'AWND', 'SNWD', 'PRCP']
powell_data[columns_to_update] = powell_data[columns_to_update].fillna(0)

powell_data.head()

,STATION,NAME,DATE,AWND,PRCP,SNWD,TMAX,TMIN,FOG,THUNDER,HAZE
0,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/1/97,0.0,0.0,0.0,95,66,0.0,0.0,0.0
1,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/2/97,0.0,0.0,0.0,93,67,0.0,0.0,0.0
2,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/3/97,0.0,0.0,0.0,90,65,0.0,0.0,0.0
3,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/4/97,0.0,0.0,0.0,94,62,0.0,0.0,0.0
4,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/5/97,0.0,0.0,0.0,89,63,0.0,0.0,0.0


In [80]:
# code to define ideal boating weather on the lake from current dataset
def define_good_boating_day(row):
    wind_condition = row['AWND'] <= 10  # Wind speed up to 10 mph
    temp_condition = 80 <= row['TMAX'] <= 105  # Max temperature between 80°F and 105°F
    rain_condition = row['PRCP'] <= 0.1  # Precipitation up to 0.1 inches
    snow_condition = row['SNWD'] <= 1  # Snow depth up to 1 inch
    return 1 if wind_condition and temp_condition and rain_condition and snow_condition else 0

#Apply this function to the dataset to create a new target variable 
powell_data['Good_Boating_Day'] = powell_data.apply(define_good_boating_day, axis=1)

powell_data.head()

,STATION,NAME,DATE,AWND,PRCP,SNWD,TMAX,TMIN,FOG,THUNDER,HAZE,Good_Boating_Day
0,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/1/97,0.0,0.0,0.0,95,66,0.0,0.0,0.0,1
1,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/2/97,0.0,0.0,0.0,93,67,0.0,0.0,0.0,1
2,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/3/97,0.0,0.0,0.0,90,65,0.0,0.0,0.0,1
3,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/4/97,0.0,0.0,0.0,94,62,0.0,0.0,0.0,1
4,USW00003162,"PAGE MUNICIPAL AIRPORT, AZ US",6/5/97,0.0,0.0,0.0,89,63,0.0,0.0,0.0,1


In [81]:
# Checking to see if the distribution of good and bad boating days
good_boating_day_dist = powell_data['Good_Boating_Day'].value_counts(normalize=True)

good_boating_day_dist

Good_Boating_Day
0    0.626264
1    0.373736
Name: proportion, dtype: float64

In [82]:
# Feature Engineering: Extracting date-related features
powell_data['DATE'] = pd.to_datetime(powell_data['DATE'])
powell_data['MONTH'] = powell_data['DATE'].dt.month
powell_data['DAY_OF_WEEK'] = powell_data['DATE'].dt.dayofweek
powell_data['YEAR'] = powell_data['DATE'].dt.year

# Selecting features and target variable
features = ['AWND', 'PRCP', 'SNWD', 'TMAX', 'TMIN', 'FOG', 'THUNDER', 'HAZE', 'MONTH', 'DAY_OF_WEEK', 'YEAR']
target = 'Good_Boating_Day'

/var/folders/tc/gxfk63z50rgcqwwzcltg1cw80000gn/T/ipykernel_4514/3488118017.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  powell_data['DATE'] = pd.to_datetime(powell_data['DATE'])


In [83]:
# Splitting the dataset
X = powell_data[features]
y = powell_data[target]

# Splitting the data into training (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [84]:
# Handling missing values
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
X_test_imputed = imputer.transform(X_test)

# Neural Network Model

In [85]:
def create_model(input_shape, units=64, activation='relu', optimizer='nadam'):
    inputs = Input(shape=input_shape)
    x = Dense(units, activation=activation)(inputs)

    # Ensuring same shape for residual connections
    for _ in range(2):  # Adjust number of layers as needed
        x = BatchNormalization()(x)
        x = Activation(activation)(x)
        x_skip = x  # Skip connection set after activation
        x = Dense(units, activation=activation)(x)
        x = Dropout(0.3)(x)
        x = add([x, x_skip])  # Residual connection

    outputs = Dense(1, activation='sigmoid')(x)  # Output layer
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [86]:
# Hyperparameter tuning variables
best_score = 0
best_params = {}
best_model_path = 'best_model_complex.h5'

# Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

In [87]:
# Iterate through hyperparameters
for units in [64, 128]:
    for activation in ['relu', 'swish']:
        for optimizer in ['adam', 'nadam']:
            model = create_model(X_train_imputed.shape[1:], units=units, activation=activation, optimizer=optimizer)
            best_model_path = 'best_model_complex.keras'  # Use .keras extension
            model_checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', save_best_only=True)
            history = model.fit(X_train_imputed, y_train, epochs=100, batch_size=32, verbose=0,
                                validation_data=(X_val_imputed, y_val),
                                callbacks=[early_stopping, reduce_lr, model_checkpoint])
            score = model.evaluate(X_val_imputed, y_val, verbose=0)[1]
            if score > best_score:
                best_score = score
                best_params = {'units': units, 'activation': activation, 'optimizer': optimizer}

print("Best parameters: ", best_params)

# Train and evaluate the best complex model
best_complex_model = keras.models.load_model(best_model_path)
accuracy = best_complex_model.evaluate(X_val_imputed, y_val)[1]
print("Accuracy on Validation Data:", accuracy)


Best parameters:  {'units': 128, 'activation': 'swish', 'optimizer': 'nadam'}
62/62 [==============================] - 0s 2ms/step - loss: 0.0668 - accuracy: 0.9857
Accuracy on Validation Data: 0.9857069849967957


# Weather Forecast

In [23]:
import requests
import json
import schedule
import time
from tensorflow.keras.models import load_model

In [24]:
# Function to get the forecast gridpoint URL
def get_forecast_gridpoint(latitude, longitude):
    url = f"https://api.weather.gov/points/{latitude},{longitude}"
    response = requests.get(url)
    if response.status_code == 200:
        grid_data = response.json()
        forecast_url = grid_data['properties']['forecast']
        return forecast_url
    else:
        print("Error getting gridpoint data:", response.status_code)
        return None

# Function to get and save the 7-day weather forecast data to a JSON file
def get_and_save_weather_forecast(forecast_url, json_file_path):
    response = requests.get(forecast_url)
    if response.status_code == 200:
        forecast_data = response.json()
        # Save the forecast data to a JSON file
        with open(json_file_path, 'w') as json_file:
            json.dump(forecast_data, json_file, indent=4)
        print(f"Weather forecast saved to {json_file_path}")
    else:
        print("Error fetching the weather forecast:", response.status_code)

# Lake Powell coordinates
latitude = 36.9147
longitude = -111.4558

# Get forecast URL and save the data
forecast_url = get_forecast_gridpoint(latitude, longitude)
if forecast_url:
    json_file_path = 'lake_powell_weather_forecast.json'  # Path to save the forecast
    get_and_save_weather_forecast(forecast_url, json_file_path)

Weather forecast saved to lake_powell_weather_forecast.json


In [25]:
def assign_image_number(short_forecast):
    forecast_keywords = {
        'Sunny': 'images/image1',
        'Clear': 'images/image2',
        'Cloudy': 'images/image3',
        'Rain': 'images/image4',
        'Thunderstorm': 'images/image5',
        'Blowing' : 'images/image6',
        'Windy' : 'images/image6'
        # Add more mappings as needed
    }
    for keyword, image_number in forecast_keywords.items():
        if keyword.lower() in short_forecast.lower():
            return image_number
    return 'default_image'  # Default image if no keyword matches

In [26]:
def process_weather_forecast(json_data, model):
    processed_data = []
    for i in range(0, len(json_data['properties']['periods']), 2):
        day_forecast = json_data['properties']['periods'][i]
        night_forecast = json_data['properties']['periods'][i + 1] if i + 1 < len(json_data['properties']['periods']) else day_forecast

        max_temp = day_forecast['temperature']
        min_temp = night_forecast['temperature']
        avg_wind_speed = calculate_average_wind_speed(day_forecast['windSpeed'], night_forecast['windSpeed'])
        haze = 1 if "haze" in day_forecast['detailedForecast'].lower() else 0
        thunderstorm = 1 if "thunderstorm" in day_forecast['detailedForecast'].lower() else 0

        model_input = prepare_model_input(max_temp, min_temp, avg_wind_speed, haze, thunderstorm)
        optimal_boating_day = bool(model.predict(model_input)[0][0] > 0.5)  # Convert to Python boolean

        short_forecast = day_forecast['shortForecast']
        image_number = assign_image_number(short_forecast)

        processed_data.append({
            'date': day_forecast['startTime'].split('T')[0],
            'max_temp': max_temp,
            'min_temp': min_temp,
            'avg_wind_speed': avg_wind_speed,
            'haze': haze,
            'thunderstorm': thunderstorm,
            'short_forecast': short_forecast,
            'imageURL': image_number,
            'optimal_boating_day': optimal_boating_day
        })
    return processed_data

def calculate_average_wind_speed(day_wind, night_wind):
    day_wind_speed = extract_wind_speed(day_wind)
    night_wind_speed = extract_wind_speed(night_wind)
    return (day_wind_speed + night_wind_speed) / 2

def extract_wind_speed(wind_speed_str):
    speeds = [int(s) for s in wind_speed_str.split() if s.isdigit()]
    return sum(speeds) / len(speeds) if speeds else 0

def prepare_model_input(max_temp, min_temp, avg_wind_speed, haze, thunderstorm):
    # Placeholder values for missing features
    awnd = prcp = snwd = fog = month = day_of_week = year = 0

    # The array now has 11 features as expected by the model
    return np.array([[awnd, prcp, snwd, max_temp, min_temp, fog, thunderstorm, haze, month, day_of_week, year]])



In [27]:
import os
import json

def save_updated_forecast_to_json(data, file_path):
    try:
        if data:  # Check if the data is not empty
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=4)
            print(f"Updated forecast saved successfully to {file_path}")
        else:
            print("Warning: No data to write to file.")
    except Exception as e:
        print(f"Failed to write to {file_path}: {e}")

def fetch_and_process_weather_data():
    # Assuming get_forecast_gridpoint and other required functions are defined above
    latitude = 36.9147
    longitude = -111.4558
    forecast_url = get_forecast_gridpoint(latitude, longitude)
    
    if forecast_url:
        json_file_path = 'lake_powell_weather_forecast.json'
        get_and_save_weather_forecast(forecast_url, json_file_path)

        try:
            with open(json_file_path, 'r') as file:
                json_data = json.load(file)
        except Exception as e:
            print(f"Error loading JSON data from {json_file_path}: {e}")
            return  # Stop execution if data loading fails

        model = load_model('best_model_complex.keras')
        processed_forecast = process_weather_forecast(json_data, model)
        
        # Make sure the path is correct
        updated_json_file_path = '/Users/liamreid/Desktop/SPRING_2024_Classes/DATA_6900/powell/util/lake_powell_weather.json'
        save_updated_forecast_to_json(processed_forecast, updated_json_file_path)

if __name__ == "__main__":
    fetch_and_process_weather_data()


Weather forecast saved to lake_powell_weather_forecast.json
1/1 [==============================] - 0s 21ms/step
Updated forecast saved successfully to /Users/liamreid/Desktop/SPRING_2024_Classes/DATA_6900/powell/util/lake_powell_weather.json
